In [4]:

import pandas as pd
import numpy as np
import multiprocessing as mp
from pykrx import stock

## ETF 종목 가져오기
- 목표 : 최하위 ETF 종목 선택하기

In [5]:
# 날짜 설정 
start_date = '20220101'
end_date = '20241231'

# 모든 ETF 티커 리스트 가져오기
etf_list = stock.get_etf_ticker_list()
etf_name = [stock.get_etf_ticker_name(etf) for etf in etf_list]

In [9]:
df_etf

,495710,466810,457930,487750,445690,465780,442260,0001P0,159800,472840,...,195980,433250,494220,470310,476000,491700,0000Y0,215620,391670,472920
날짜,,,,,,,,,,,,,,,,,,,,,
2022-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30305,NaN,...,11875,NaN,NaN,NaN,NaN,NaN,NaN,12245,9710,NaN
2022-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30520,NaN,...,11875,NaN,NaN,NaN,NaN,NaN,NaN,12160,9715,NaN
2022-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,29820,NaN,...,11735,NaN,NaN,NaN,NaN,NaN,NaN,12200,9635,NaN
2022-01-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,29910,NaN,...,11635,NaN,NaN,NaN,NaN,NaN,NaN,12135,9475,NaN
2022-01-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,29910,NaN,...,11705,NaN,NaN,NaN,NaN,NaN,NaN,12245,9580,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-12-23,50080.0,5145.0,10080.0,7000.0,13245.0,107490.0,11510.0,NaN,25080,32380.0,...,9780,10630.0,8620.0,7130.0,6145.0,32405.0,NaN,12935,8150,109145.0
2024-12-24,50085.0,5095.0,10060.0,6995.0,13280.0,107485.0,11540.0,9710.0,25120,32430.0,...,9800,10615.0,8520.0,7055.0,6205.0,32405.0,100020.0,12960,8170,108960.0
2024-12-26,50080.0,4965.0,10015.0,6880.0,13235.0,107470.0,11505.0,9715.0,24970,32240.0,...,9820,10555.0,8550.0,7060.0,6160.0,32240.0,100035.0,12920,8145,108750.0


In [10]:
# 각 ETF 종목의 평균 종가 계산
etf_avg_prices = df_etf.mean()

# 평균 종가가 가장 낮은 1 개의 종목 찾기
lowest_etfs = etf_avg_prices.nsmallest(1) 

# 결과 출력
print("최하위 ETF 종목:")
print(lowest_etfs)

최하위 ETF 종목:
204450    2118.390476
dtype: float64


In [22]:

# 최하위 ETF 종목 (예: '204450')
lowest_etf_ticker = '204450'

# ETF 구성 종목과 비중 가져오기
portfolio = stock.get_etf_portfolio_deposit_file(lowest_etf_ticker, date='20240221')  # 현재 날짜 기준

# 결과 확인
print("ETF 구성 종목과 비중:")
print(portfolio)

# 데이터프레임 구조 확인 (종목 코드, 종목명, 비중 등 포함)
print(portfolio.columns)

ETF 구성 종목과 비중:
                 계약수  금액   비중
티커                           
100000  2.543600e+03   0  0.0
017191  6.141000e+02   0  0.0
040111  1.172500e+02   0  0.0
070341  5.853000e+01   0  0.0
100000  6.920220e+03   0  0.0
100000  7.235600e+02   0  0.0
100000  8.627000e+01   0  0.0
100000  8.951760e+03   0  0.0
100000  6.159600e+02   0  0.0
941009  5.083500e+02   0  0.0
100000  2.792200e+02   0  0.0
100000  3.228200e+02   0  0.0
883013  1.291280e+03   0  0.0
000330  2.819332e+04   0  0.0
093012  7.384000e+02   0  0.0
210961  2.616000e+02   0  0.0
688002  3.172500e+02   0  0.0
291001  1.335800e+02   0  0.0
2108Y1  2.671600e+02   0  0.0
267001  6.011100e+02   0  0.0
3066L1  6.549000e+01   0  0.0
3777B1  4.990700e+02   0  0.0
4290A1  1.391500e+02   0  0.0
100004  2.011100e+02   0  0.0
4INDEX  2.630000e+00   0  0.0
100000  6.103900e+03   0  0.0
5074A1  9.212000e+01   0  0.0
5074S1  2.187400e+02   0  0.0
8208B1  1.447100e+02   0  0.0
532631  2.523200e+02   0  0.0
5479M1  1.139100e+02   0 